In [ ]:
# === Cell 1: Unified Environment & Project-Wide Setup ===
import os, json, math, datetime as dt
from datetime import datetime  # for human-readable prints
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional (map preview): keep if you use geemap
try:
    import geemap
    GEEMAP_AVAILABLE = True
except Exception:
    GEEMAP_AVAILABLE = False

plt.style.use('seaborn-v0_8-whitegrid')
os.makedirs('outputs', exist_ok=True)

import ee

# --- Earth Engine authentication and initialization ---
# You can override with: export EE_PROJECT_ID=your-project-id
EE_PROJECT_ID = os.environ.get('EE_PROJECT_ID', 'nasa-flood')  # <— 원하는 기본 프로젝트 ID
def _ee_init():
    try:
        ee.Initialize()
        return "Initialized with default credentials"
    except Exception:
        try:
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Initialized with project='{EE_PROJECT_ID}'"
        except Exception:
            print("Authenticating with Earth Engine...")
            ee.Authenticate()
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Authenticated & initialized with project='{EE_PROJECT_ID}'"

print(_ee_init())
print(f"Current time: {datetime.now().isoformat(timespec='seconds')}")

# ===== Project-wide constants =====
CFG = {
    # AOIs (EPSG:4326)
    # Mekong Delta (VN lower basin); box chosen for stability/reproducibility
    "AOI_DELTA": ee.Geometry.Rectangle([104.30,  8.50, 106.90, 10.90], geodesic=False),
    # Tonlé Sap (KH); "Mekong's heartbeat"
    "AOI_TONLESAP": ee.Geometry.Rectangle([103.30, 12.00, 105.20, 13.70], geodesic=False),

    # Analysis windows
    "YEARS": list(range(2015, 2025)),
    "FLOOD_MONTHS": (8, 9),     # Aug–Sep (wet-season peak)
    "DROUGHT_MONTHS": (3, 4),   # Mar–Apr (dry-season trough)

    # Thresholds (physical/empirical, fixed for comparability)
    "TH_VV_DB": -16.0,
    "TH_VH_DB": -22.0,

    # Landsat5 baseline window (pre-major-dam reference)
    "BASELINE_YEARS": [2005, 2006, 2007, 2008],

    # Event markers for plots
    "EVENTS": {
        "JINGHONG_FLOW_CUT": "2019-07-15",  # Jinghong flow cut (smoking gun)
        "XIAOWAN_ONLINE":    "2009-01-01",
        "NUOZHADU_ONLINE":   "2012-01-01"
    }
}

# ===== Utilities for Earth Engine =====
def _daterange_of_year_months(year:int, m1:int, m2:int):
    """Return ISO start and inclusive end-of-month last day for [m1..m2]."""
    start = dt.date(year, m1, 1)
    if m2 == 12:
        end = dt.date(year+1, 1, 1) - dt.timedelta(days=1)
    else:
        end = dt.date(year, m2+1, 1) - dt.timedelta(days=1)
    return start.isoformat(), end.isoformat()

def s1_min(aoi, start, end, pol):
    """Min-composite Sentinel-1 GRD over period to stabilize speckle."""
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(aoi)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode','IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
            .select(pol)
            .min()
            .clip(aoi))

def classify_water(img_min, pol, threshold_db):
    """Water = backscatter < threshold (in dB); returns self-masked binary."""
    return img_min.lt(threshold_db).selfMask()

def area_km2(mask_img, aoi, scale=30, band_name='constant'):
    """Compute km² of a self-masked image."""
    area = (mask_img.multiply(ee.Image.pixelArea())
            .reduceRegion(ee.Reducer.sum(), aoi, scale, maxPixels=1e12))
    # selfMask() produces band 'constant'
    return ee.Number(area.get(band_name)).divide(1e6)

def landsat5_c2_sr_mask_scale(img):
    """L5 C2 L2 scaling + cloud/shadow masking."""
    qa = img.select('QA_PIXEL')
    cloud  = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(shadow).eq(0))
    optical = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = img.select('ST_B6').multiply(0.00341802).add(149.0)
    return (img.addBands(optical, None, True)
               .addBands(thermal, None, True)
               .updateMask(mask))

def landsat5_median(aoi, start_date, end_date, months=None):
    """Median composite with optional wrapped-month filter ((11,12),(1,4))."""
    col = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
           .filterBounds(aoi)
           .filterDate(start_date, end_date)
           .map(landsat5_c2_sr_mask_scale))
    if months:
        # e.g., months=((11,12),(1,4)) to handle dry-season wrap
        f = ee.Filter.Or(
            ee.Filter.calendarRange(int(months[0][0]), int(months[0][1]), 'month'),
            ee.Filter.calendarRange(int(months[1][0]), int(months[1][1]), 'month')
        )
        col = col.filter(f)
    return col.median().clip(aoi)

def mndwi(img):  # SR_B2 (Green), SR_B5 (SWIR)
    return img.normalizedDifference(['SR_B2','SR_B5']).rename('MNDWI')

def water_mask_from_mndwi(img, threshold=0.0):
    return img.gt(threshold).selfMask()

def chirps_sum_mm(aoi, start, end):
    """Return AOI-mean of CHIRPS precipitation sum (mm) over [start,end]."""
    col = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
           .filterBounds(aoi).filterDate(start, end).select('precipitation'))
    total = col.sum().reduceRegion(ee.Reducer.mean(), aoi, 5000, maxPixels=1e12)
    return ee.Number(total.get('precipitation'))

# (Optional) quick summary print
print(f"AOI_DELTA bounds: {CFG['AOI_DELTA'].bounds().getInfo()['coordinates'][0][0]} …")
print(f"AOI_TONLESAP bounds: {CFG['AOI_TONLESAP'].bounds().getInfo()['coordinates'][0][0]} …")
print("Setup complete ✅")


Earth Engine initialized successfully
Current time: 2025-10-04 17:30:57.608268


In [ ]:
# === Cell 2: Baseline (2005–2008) with Landsat5 + JRC cross-check ===
# 목적:
#  - 댐 영향 전(2005–2008)의 '우기'와 '건기' 기준선을 Landsat5로 구축
#  - JRC Global Surface Water의 seasonality(12=영구 수역)로 교차 검증
#  - DELTA / TONLE SAP 각각의 기준선 면적 수치로 저장

# 1) 기간/영역 정의
baseline_start = f"{min(CFG['BASELINE_YEARS'])}-01-01"
baseline_end   = f"{max(CFG['BASELINE_YEARS'])}-12-31"
AOI_ALL        = CFG['AOI_DELTA'].union(CFG['AOI_TONLESAP'])

# 2) Landsat5 컬렉션 준비(스케일/마스킹 적용)
l5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
      .filterBounds(AOI_ALL)
      .filterDate(baseline_start, baseline_end)
      .map(landsat5_c2_sr_mask_scale))

# 3) 우기(5–10월) / 건기(11–12, 1–4월) 합성
l5_wet  = l5.filter(ee.Filter.calendarRange(5, 10, 'month')).median().clip(AOI_ALL)
l5_dry  = l5.filter(ee.Filter.Or(
                ee.Filter.calendarRange(11, 12, 'month'),
                ee.Filter.calendarRange(1, 4, 'month')
            )).median().clip(AOI_ALL)

# 4) MNDWI 기반 수역 마스크
mndwi_wet = mndwi(l5_wet)
mndwi_dry = mndwi(l5_dry)
water_wet_mask = water_mask_from_mndwi(mndwi_wet, threshold=0.0).rename('water')
water_dry_mask = water_mask_from_mndwi(mndwi_dry, threshold=0.0).rename('water')

# 5) JRC 영구 수역(= seasonality == 12) 교차검증 마스크
jrc = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').clip(AOI_ALL)
jrc_perm = jrc.select('seasonality').eq(12).selfMask().rename('perm')

# 6) 면적 계산 함수(EE Number -> float)
def _num(x): 
    return float(ee.Number(x).getInfo()) if x is not None else None

def aoi_areas(name, aoi):
    wet_km2  = _num(area_km2(water_wet_mask, aoi))     # 우기 기준선
    dry_km2  = _num(area_km2(water_dry_mask, aoi))     # 건기 기준선
    perm_km2 = _num(area_km2(jrc_perm, aoi, band_name='perm'))  # JRC 영구수역
    return {"aoi": name, "baseline_wet_km2": wet_km2, "baseline_dry_km2": dry_km2, "jrc_perm_km2": perm_km2}

summary_rows = [
    aoi_areas("Mekong_Delta", CFG['AOI_DELTA']),
    aoi_areas("Tonle_Sap",    CFG['AOI_TONLESAP']),
    aoi_areas("Union_ALL",    AOI_ALL),
]
df_baseline = pd.DataFrame(summary_rows)
display(df_baseline)

# 7) 산출물 저장
os.makedirs('outputs', exist_ok=True)
df_baseline.to_csv('outputs/baseline_areas.csv', index=False)
with open('outputs/baseline_summary.json','w', encoding='utf-8') as f:
    json.dump({
        "baseline_years": CFG['BASELINE_YEARS'],
        "wet_months": [5,10],
        "dry_months": [[11,12],[1,4]],
        "areas": summary_rows
    }, f, ensure_ascii=False, indent=2)

print("Saved:",
      "outputs/baseline_areas.csv",
      "outputs/baseline_summary.json", sep="\n- ")

# 8) (선택) GeoTIFF로 로컬 저장 — 발표/데모용 레이어
if GEEMAP_AVAILABLE:
    geemap.ee_export_image(mndwi_wet,  filename='outputs/mndwi_wet_2005_2008.tif',  scale=30, region=AOI_ALL, file_per_band=False)
    geemap.ee_export_image(mndwi_dry,  filename='outputs/mndwi_dry_2005_2008.tif',  scale=30, region=AOI_ALL, file_per_band=False)
    geemap.ee_export_image(water_wet_mask, filename='outputs/watermask_wet_2005_2008.tif', scale=30, region=AOI_ALL, file_per_band=False)
    geemap.ee_export_image(water_dry_mask, filename='outputs/watermask_dry_2005_2008.tif', scale=30, region=AOI_ALL, file_per_band=False)
    geemap.ee_export_image(jrc_perm,    filename='outputs/jrc_perm_seasonality12.tif',     scale=30, region=AOI_ALL, file_per_band=False)
    print("GeoTIFF exports done.")

# 9) (선택) 지도 미리보기
if GEEMAP_AVAILABLE:
    m = geemap.Map(center=[12.1, 105.2], zoom=7)
    m.addLayer(l5_wet.select(['SR_B3','SR_B2','SR_B1']), {'min':0, 'max':0.3}, 'L5 Wet RGB')
    m.addLayer(mndwi_wet, {'min':-1, 'max':1, 'palette':['#8c510a','#f6e8c3','#35978f','#01665e']}, 'MNDWI Wet')
    m.addLayer(water_wet_mask, {'palette':['#1f78b4']}, 'Water Mask (Wet)')
    m.addLayer(jrc_perm, {'palette':['#e31a1c']}, 'JRC Permanent Water (seasonality=12)')
    m.addLayerControl()
    m


Study Area: Vietnam Mekong Delta
Total Area: 70,071.10 km²
